In [193]:
#Import packages
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import STOPWORDS


In [194]:
# Adding multiple words to gensim stoplist
new_stopword = STOPWORDS.union(set(['a', 'the','an']))

In [195]:
from tqdm import tqdm
import numpy as np
import gensim # thư viện NLP
import os 
from nltk.tokenize import word_tokenize
doc = []

In [196]:
dir_path = os.path.dirname(os.path.realpath(os.getcwd()))
data_path = os.path.join(dir_path, 'data\\train_set_ielts')
test_path = os.path.join(dir_path, 'data\\test_set_ielts')

In [197]:
data_path
test_path

'D:\\Projects\\Python\\topic-classification\\data\\test_set_ielts'

In [198]:
def rm_stopword(tokenized_doc,stop_words):
    tok_without_sw=[]
    for txt_tokens in tokenized_doc:
        tok_without_sw = [word for word in tokenized_doc if not word.lower() in STOPWORDS]
    return tok_without_sw

In [199]:
def get_data(folder_path):
    dirs = os.listdir(folder_path)
    processed_doc = []
    for path in tqdm(dirs):
        file_paths = os.listdir(os.path.join(folder_path, path))
        for file_path in tqdm(file_paths):
            with open(os.path.join(folder_path, path, file_path), 'r') as f:
                lines = f.readlines()
                lines = ' '.join(lines)
                lines = gensim.utils.simple_preprocess(lines)
                lines = ' '.join(lines)

                # Tokenization of each document
                tokenized_doc = word_tokenize(lines.lower())
                #remove stop word
                tokenized_doc = rm_stopword(tokenized_doc,new_stopword)
                processed_doc.append([tokenized_doc,path])
    return processed_doc

# def get_data_to_test(folder_path):
#     dirs = os.listdir(folder_path)
#     test_doc = []
#     for path in tqdm(dirs):
#         file_paths = os.listdir(os.path.join(folder_path, path))
#         for file_path in tqdm(file_paths):
#             with open(os.path.join(folder_path, path, file_path), 'r') as f:
#                 lines = f.readlines()
#                 lines = ' '.join(lines)
#                 lines = gensim.utils.simple_preprocess(lines)
#                 lines = ' '.join(lines)
#                 tokenized_doc = word_tokenize(lines.lower())
#                 tokenized_doc = rm_stopword(tokenized_doc,new_stopword)
#                 test_doc.append([tokenized_doc, path])
#     return test_doc

def run_test(model, test_doc):
    test_pass = 0
    for index in range(len(test_doc)):
        result = model.docvecs.most_similar(positive=[model.infer_vector(test_doc[index][0])],topn=6)
        if (result[0][0] == test_doc[index][1]):
            test_pass+= 1
    return [test_pass,len(test_doc)]

In [200]:
data_train = get_data(data_path)
data_test = get_data(test_path)

100%|██████████| 6/6 [00:00<00:00, 22.96it/s]


In [201]:
len(data_train)

56

In [202]:
len(data_test)

17

In [203]:
tagged_doc =[]
for case in range(len(data_train)):
    case_i = TaggedDocument(data_train[case][0],[data_train[case][1]])
    tagged_doc.append(case_i)

In [204]:
import multiprocessing
cores = multiprocessing.cpu_count()

In [205]:
# Train doc2vec model
model = Doc2Vec(tagged_doc, vector_size=300, window=2, min_count=1, workers=cores, epochs = 100)

In [206]:
# Save trained doc2vec model
model_path = os.path.dirname(os.path.realpath(os.getcwd()))
model_path = os.path.join(model_path, 'models\\model_doc2vec_ielts.model')
model.save(model_path)

In [207]:
# Load saved doc2vec model
model= Doc2Vec.load(model_path)

In [211]:
# Test
pass_percent = run_test(model, data_test)
pass_percent

[10, 17]

In [ ]:
# find most similar doc topic

text_check = """
Some believe children should be taught to give speeches and presentations in school.

Why is this?

Should this be taught in schools?

Many believe that giving presentations and speeches ought to be a key part of school curricula for children. In my opinion, this is an attempt to develop a number of skills holistically and should be encouraged.

The main reason schools incorporate presentations is to improve skills needed for the future. Children will later be expected to present individually and in groups throughout their academic career and in most work contexts. In order to give a competent presentation, children must first of all develop confidence speaking in front of others and then combine this with careful preparation, repeated practice, research, and, often, team-working skills. Each of these qualities will be useful later and the earlier students begin, the more likely they are to excel in areas that many adults still find challenging.

I would recommend this practice continues since integration of skills contributes to greater progress. Skills developed on their own are often not as memorable. If a young child, for example, must do a book report with a group of other children this requires them to read the book, divide up sections of the presentation, communicate with team members, and deliver an engaging speech at the end. The combination of all skills makes the learning more memorable and likely to develop fixed characteristics. An illustrative analogy would be how an athlete practices for a sport. They can master individual skills on their own but the greatest progress comes when they blend them under the intense pressure of a game.

In conclusion, educators often teach public speaking in order to prepare students for the future and this multidisciplinary approach is a positive. Presentations and speeches are also a good way to combine and review past lessons.
"""

In [144]:
test_doc = word_tokenize(text_check.lower())
result = model.docvecs.most_similar(positive=[model.infer_vector(test_doc)],topn=6)
result[0][0]

'education'